In [1]:
import tensorflow as tf

# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
# set parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

# network parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [3]:
weights = dict()
biases  = dict()

def multilayer_perceptron(x):
    with tf.variable_scope("layer_1"):
        weights["w1"] = tf.Variable(tf.random_normal([n_input, n_hidden_1]))
        biases["b1"]  = tf.Variable(tf.random_normal([n_hidden_1]))
        layer_1 = tf.add(tf.matmul(x, weights["w1"]), biases["b1"])
        layer_1 = tf.nn.relu(layer_1)
    
    with tf.variable_scope("layer_2"):
        weights["w2"] = tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]))
        biases["b2"]  = tf.Variable(tf.random_normal([n_hidden_2]))
        layer_2 = tf.add(tf.matmul(layer_1, weights["w2"]), biases["b2"])
        layer_2 = tf.nn.relu(layer_2)
    
    with tf.variable_scope("out"):
        weights["out"] = tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
        biases["out"]  = tf.Variable(tf.random_normal([n_classes]))
        out_layer = tf.matmul(layer_2, weights["out"]) + biases["out"]
        return out_layer

with tf.name_scope("netowrk"):
    pred = multilayer_perceptron(x)
    
with tf.name_scope('loss'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
    
with tf.name_scope('SGD'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
    
with tf.name_scope('accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))

In [4]:
init = tf.global_variables_initializer()

# create a summary to monitor cost tensor
tf.scalar_summary("loss", cost)
# create a summary to monitor accuracy tensor
tf.scalar_summary("accuracy", acc)
# merge all summaries into a single op
merged_summary_op = tf.merge_all_summaries()

In [5]:
config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
sess = tf.Session(config=config)
sess.run(init)

# op to write logs to Tensorboard
summary_writer = tf.train.SummaryWriter("tmp/tensorflow_logs", graph=tf.get_default_graph())

# training cycle
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(mnist.train.num_examples/batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # run optimization op (backprop), cost op (to get loss value)
        # and summary nodes
        _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                 feed_dict={x: batch_xs, y: batch_ys})
        # write logs at every iteration
        summary_writer.add_summary(summary, epoch * total_batch + i)
        # compute average loss
        avg_cost += c / total_batch
    
    # display logs per epoch step
    if (epoch+1) % display_step == 0:
        print("Epoch: {0:04d} cost={1:.5f}" .format(epoch+1, avg_cost))
print("Optimization Finished!")

# Test model
# Calculate accuracy
print("Train accuracy: {0}" 
      .format(sess.run(acc, feed_dict={x: mnist.train.images, y: mnist.train.labels})))
print("Test accuracy: {0}" 
      .format(sess.run(acc, feed_dict={x: mnist.test.images, y: mnist.test.labels})))

Epoch: 0001 cost=159.86736
Epoch: 0002 cost=58.33803
Epoch: 0003 cost=42.59859
Epoch: 0004 cost=34.21207
Epoch: 0005 cost=28.90704
Epoch: 0006 cost=25.18798
Epoch: 0007 cost=22.32719
Epoch: 0008 cost=20.14069
Epoch: 0009 cost=18.28160
Epoch: 0010 cost=16.78392
Epoch: 0011 cost=15.47625
Epoch: 0012 cost=14.33692
Epoch: 0013 cost=13.32889
Epoch: 0014 cost=12.51397
Epoch: 0015 cost=11.72000
Optimization Finished!
Train accuracy: 0.932818591594696
Test accuracy: 0.9124001264572144


## Run tensorboard

```shell
$ tensorboard --logdir=tmp/tensorflow_logs --host=... --port=...
```

![graph](../assets/graph.png)
![loss](../assets/loss.png)

### BLACK???????
Actually, in tensorflow 0.11 version there are some bugs when plot scalr graph... :(<br>
See https://github.com/tensorflow/tensorflow/issues/4856